In [2]:
from pennylane import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt

from functions_vqe_crx import vqe_hee, vqe_ng
import numpy as np
from joblib import Parallel, delayed

device = 'cpu'

## AntiF Heisenberg hamiltonian definition 

In [3]:
n_cells = (2, 3)
coupling = [1,1,1]

qubits = n_cells[0] * n_cells[1]

lattice = 'triangle'

H = qml.spin.heisenberg(lattice, n_cells, coupling, boundary_condition=True)

H

(
    X(3) @ X(5)
  + Y(3) @ Y(5)
  + Z(3) @ Z(5)
  + X(2) @ X(5)
  + Y(2) @ Y(5)
  + Z(2) @ Z(5)
  + X(3) @ X(4)
  + Y(3) @ Y(4)
  + Z(3) @ Z(4)
  + X(2) @ X(3)
  + Y(2) @ Y(3)
  + Z(2) @ Z(3)
  + X(0) @ X(3)
  + Y(0) @ Y(3)
  + Z(0) @ Z(3)
  + X(4) @ X(5)
  + Y(4) @ Y(5)
  + Z(4) @ Z(5)
  + X(0) @ X(4)
  + Y(0) @ Y(4)
  + Z(0) @ Z(4)
  + X(1) @ X(4)
  + Y(1) @ Y(4)
  + Z(1) @ Z(4)
  + X(1) @ X(5)
  + Y(1) @ Y(5)
  + Z(1) @ Z(5)
  + X(0) @ X(2)
  + Y(0) @ Y(2)
  + Z(0) @ Z(2)
  + X(0) @ X(1)
  + Y(0) @ Y(1)
  + Z(0) @ Z(1)
  + X(0) @ X(5)
  + Y(0) @ Y(5)
  + Z(0) @ Z(5)
  + X(1) @ X(2)
  + Y(1) @ Y(2)
  + Z(1) @ Z(2)
  + X(1) @ X(3)
  + Y(1) @ Y(3)
  + Z(1) @ Z(3)
  + X(2) @ X(4)
  + Y(2) @ Y(4)
  + Z(2) @ Z(4)
)

## Distribution of results

### Parameters

In [4]:
max_iterations = 200

layers = np.arange(2, 27, 4)
n_layers = len(layers)

n_runs = 100

conv_tol = 1e-06

weights_lr = 0.01
phi_lr = 0.01

In [5]:
prefix1 = "../local_data/log5/"
spec1 = "_crx_1"

prefix2 = prefix1
spec2 = "_crx_2"

In [6]:
def run_single_simulation_1(i, l, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_hee(H, qubits, l, lr=weights_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)
    
    return i, j, (energy, grad_norm, grad_var, conv)

def run_single_simulation_2(i, l, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_ng(H, qubits, l, weights_lr=weights_lr, phi_lr=phi_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)

    return i, j, (energy, grad_norm, grad_var, conv)

### HEE

In [27]:
tasks = (delayed(run_single_simulation_1)(i, l, j)
         for i, l in enumerate(layers)
         for j in range(n_runs))
print('number of tasks:', n_layers * n_runs)
results = Parallel(n_jobs=20, verbose=11)(tasks)

number of tasks: 700


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done   2 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:    4.5s
[Parallel(n_jobs=20)]: Done   4 tasks      | elapsed:    4.6s
[Parallel(n_jobs=20)]: Done   5 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done   7 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done   9 tasks      | elapsed:    6.0s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    6.1s
[Parallel(n_jobs=20)]: Done  11 tasks      | elapsed:    6.2s
[Parallel(n_jobs=20)]: Done  12 tasks      | elapsed:    6.2s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:    6.2s
[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:    6.2s
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  

#### Save

In [28]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [29]:
np.save(prefix1+"energies"+spec1+".npy", energies)
np.save(prefix1+"grad_norms"+spec1+".npy", grad_norms)
np.save(prefix1+"grad_variances"+spec1+".npy", grad_variances)
np.save(prefix1+"convergences"+spec1+".npy",convergences)

### NG

In [7]:
tasks = (delayed(run_single_simulation_2)(i, l, j)
         for i, l in enumerate(layers)
         for j in range(n_runs))
print('number of tasks:', n_layers * n_runs)
results = Parallel(n_jobs=20, verbose=11)(tasks)

number of tasks: 700


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    4.1s
[Parallel(n_jobs=20)]: Done   2 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done   4 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done   5 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:    4.5s
[Parallel(n_jobs=20)]: Done   7 tasks      | elapsed:    4.5s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:    4.6s
[Parallel(n_jobs=20)]: Done   9 tasks      | elapsed:    4.7s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    4.7s
[Parallel(n_jobs=20)]: Done  11 tasks      | elapsed:    4.7s
[Parallel(n_jobs=20)]: Done  12 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:    5.0s
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  

#### Save

In [8]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [9]:
np.save(prefix2+"energies"+spec2+".npy", energies)
np.save(prefix2+"grad_norms"+spec2+".npy", grad_norms)
np.save(prefix2+"grad_variances"+spec2+".npy", grad_variances)
np.save(prefix2+"convergences"+spec2+".npy",convergences)